In [1]:
import os
import re
import json
import random
import collections
from pprint import pprint
from pathlib import Path
import sys

import numpy as np
import pandas as pd
import scipy

from pprint import pprint
from mne_bids import read_raw_bids, BIDSPath, get_entity_vals
from natsort import natsorted, index_natsorted, order_by_index
import scipy
from scipy import interp
from sklearn import preprocessing

from sklearn import random_projection

from eztrack.fragility.linearsystem import DiscreteLinearSystem
from eztrack.fragility.perturbationmodel import MinNormPerturbModel

sys.path.append('../')

from analysis.fragility.posthoc import read_perturbation_result, run_svd_viz

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# Load Data

In [2]:
WORKSTATION = "home"

if WORKSTATION == "home":
    # bids root to write BIDS data to
    # the root of the BIDS dataset
    root = Path("/Users/adam2392/Dropbox/epilepsy_bids/")
    output_dir = root / 'derivatives' / 'interictal'

    figures_dir = output_dir / 'figures'

    # path to excel layout file - would be changed to the datasheet locally
    excel_fpath = Path(
        "/Users/adam2392/Dropbox/epilepsy_bids/sourcedata/organized_clinical_datasheet_raw.xlsx"
    )
elif WORKSTATION == "lab":
    root = Path("/home/adam2392/hdd/epilepsy_bids/")
    excel_fpath = Path(
        "/home/adam2392/hdd/epilepsy_bids/sourcedata/organized_clinical_datasheet_raw.xlsx"
    )

    # output directory
    output_dir = Path("/home/adam2392/hdd/epilepsy_bids") / 'derivatives' / 'interictal'

    # figures directory
    figures_dir = output_dir / 'figures'

figures_dir = root / 'derivatives' / 'rowvscol'
figures_dir.mkdir(exist_ok=True, parents=True)

In [3]:
SUBJECTS = [
#     'jh103',
#     'jh105'
    'pt1'
]

session = "presurgery"  # only one session
task = "interictal"
datatype = "ieeg"
acquisition = "ecog"  # or SEEG
extension = ".vhdr"

if acquisition == 'ecog':
    ignore_acquisitions = ['seeg']
elif acquisition == 'seeg':
    ignore_acquisitions = ['ecog']

reference = 'monopolar'
sfreq = None  # either resample or don't

# get the runs for this subject
all_subjects = get_entity_vals(root, "subject")

for subject in all_subjects:
    if subject not in SUBJECTS:
        continue
    ignore_subs = [sub for sub in all_subjects if sub != subject]
    all_tasks = get_entity_vals(root, "task", ignore_subjects=ignore_subs)
    ignore_tasks = [tsk for tsk in all_tasks if tsk != task]

    print(f"Analyzing {task} task for {subject}.")
    ignore_tasks = [tsk for tsk in all_tasks if tsk != task]
    runs = get_entity_vals(
        root, 'run', ignore_subjects=ignore_subs,
        ignore_tasks=ignore_tasks,
        ignore_acquisitions=ignore_acquisitions
    )
    print(f'Found {runs} runs for {task} task.')

    deriv_path = (output_dir
                  # /  'nodepth'
                  / '1000Hz'
                  / "fragility"
                  / reference
                  / f"sub-{subject}")
    
    for idx, run in enumerate(runs):
        # create path for the dataset
        bids_path = BIDSPath(
            subject=subject,
            session=session,
            task=task,
            run=run,
            datatype=datatype,
            acquisition=acquisition,
            suffix=datatype,
            root=root,
#             extension=extension,
        )
        source_basename = bids_path.basename
        description = 'statematrix'
        result = read_perturbation_result(deriv_path, source_basename, description)
        A_mats, result_info, sidecar_json = result[0]
        
        pat_dict = read_clinical_excel(excel_fpath, subject=subject)
        
        # extract the SOZ channels
        soz_chs = pat_dict[ClinicalContactColumns.SOZ_CONTACTS.value]
        epz_chs = pat_dict[ClinicalContactColumns.SPREAD_CONTACTS.value]
        rz_chs = pat_dict[ClinicalContactColumns.RESECTED_CONTACTS.value]

        break
    break

Analyzing interictal task for pt1.
Found ['01', '02', '03', '04'] runs for interictal task.
TRYING TO READ RESULT FILENAME  sub-pt1_ses-presurgery_task-interictal_acq-ecog_run-01_desc-statematrix_ieeg.npy
Loading /Users/adam2392/Dropbox/epilepsy_bids/derivatives/interictal/1000Hz/fragility/monopolar/sub-pt1/sub-pt1_ses-presurgery_task-interictal_acq-ecog_run-01_desc-statematrix_ieeg.npy, which exists: True


ValueError: too many values to unpack (expected 3)

# Compute Transition Probability Matrix Instead

In [3]:
model_kwargs = {
    'radius': 1.05,
    'perturb_type': 'C',
    'method_to_use': 'dc',
}
pert_model = MinNormPerturbModel(**model_kwargs)

In [5]:
mean = 0
variance = 1
m = 100

# generate sub-gaussian matrix with independent entries
A = np.random.normal(loc=mean, scale=variance, size=(m, m))
A_symm = (A + A.T) / 2

# generate data w/ it
dlds = DiscreteLinearSystem(A=A_symm)

In [6]:
data = dlds.reconstruct(x0=np.ones((m,)))